In [1]:
#-----------------------------------------
# importamos los paquetes que vamos a usar
#-----------------------------------------

import pypulse as pulsar
import matplotlib.pyplot as plt     # para graficar

import numpy as np
import glob                         # para hacer listas de archivo
import subprocess                   # para usar subprocesos

import os
import shutil

import decimal

In [2]:
#----------------------------------
# Cargamos las observaciones
#----------------------------------

timing_folder = './pfds/'              # nombre del directorio que contiene los .pfds
os.chdir(timing_folder)                            # entramos al directorio que contiene los .pfds
pfds = glob.glob('*pfd')                           # creamos una lista que contiene los .pfds
print("Observaciones: " + str(len(pfds)))
    
os.chdir('..')                                     # salimos del directorio que contiene los .pfds
    
# cargamos el archivo que contiene los t_obs

data = "./results_total_latest.txt"

files = np.loadtxt( data , skiprows=0 , dtype=str , usecols=(0) )
sn_obs = np.loadtxt( data , skiprows=0 , dtype=str , usecols=(1) )
t_obs = np.loadtxt( data , skiprows=0 , dtype=str , usecols=(2) )

Observaciones: 403


In [3]:
#t_duracion_min = 70
t_duracion_min = 0
#sn_min = 50
sn_min = 0
criterio_sigma_toas = 0.70
#criterio_sigma_toas = 10000

template = "J0437-4715.pfd.std"

In [4]:
#----------------------------------------------------------------------------
# Calculamos TOAs
#----------------------------------------------------------------------------           

# creamos una lista con todos los archivos .pfd de observaciones

dirName = './pfds/'

os.chdir(dirName)                                       # entramos al directorio que contiene los .pfds
pfd_files = glob.glob('*pfd')                           # lista de archivos .pfd
print("Total de observaciones = " + str(len(pfd_files)))

# ahora llamamos a pat para generar el archivo .tim

tim_temp = "timing_temp.tim"    # nombre del archivo .tim temporal que usaremos al ir barriendo las observaciones
tim_out = "timing_tmin" + str(t_duracion_min) + "_snmin" + str(sn_min) + "_crit" + str(criterio_sigma_toas*100) + ".tim"

f = open(tim_out, 'w+')                                               # abrimos el nuevo archivo .tim
f.write("FORMAT 1\n")
f.close()

j=0
k=0

# Recorremos cada una de las observaciones y vemos que no sea un outlier y que satisfaga los criterios dados

for pfd in pfd_files:
    if  float(t_obs[ np.where(files == pfd)[0][0] ]) >= t_duracion_min \
    and float(sn_obs[ np.where(files == pfd)[0][0] ]) >= sn_min \
    and pfd != "prepfold_timing_A2_20200529_110950_PSR_0437-4715.pfd" \
    and pfd != "prepfold_timing_A2_20200525_112535_PSR_0437-4715.pfd" \
    and pfd != "prepfold_timing_A2_20200525_123618_PSR_0437-4715.pfd" \
    and pfd != "prepfold_timing_A2_20200504_124809_PSR_0437-4715.pfd" \
    and pfd != "prepfold_timing_A2_20200507_135407_PSR_0437-4715.pfd" \
    and pfd != "prepfold_timing_A2_20200421_133912_PSR_0437-4715.pfd": # pedimos que las observaciones duren al menos t_duracion_min y nos deshacemos de los outliers         

# Calculamos los TOAs de esta observación particular
    
        subprocess.call("pat -FT -A PGS -s " + template + " -f >" + tim_temp + " 'tempo2' " + pfd, shell = True)
        
# Leemos los datos de los TOAs de esta observación particular

        file = np.loadtxt( tim_temp , skiprows=1 , dtype=str , usecols=(0) )             # nombre de cada observación
        freq = np.loadtxt( tim_temp, skiprows=1, dtype=decimal.Decimal , usecols=(1) )             # frecuencia de la observación
        MJD = np.loadtxt( tim_temp , skiprows=1 , dtype=decimal.Decimal , usecols=(2) )  # fecha juliana modificada de la observación
        err = np.loadtxt( tim_temp , skiprows=1 , dtype=float,  usecols=(3) )            # error en el TOA
        telescope = np.loadtxt( tim_temp, skiprows=1, dtype=str, usecols=(4) )           # antena usada (A1 o A2)     
    
# Si el error es menor al criterio utilizado, entonces escribimos los datos en el archivo .tim de salida

        if err < criterio_sigma_toas:
            with open(tim_out, "a") as f:
                f.write( str(file) + ' ' + str(freq) + ' ' + str(MJD) + '   ' + str(err) + '  ' + str(telescope) + '\n')        
                
            j += 1
            
        else:
            k += 1
                
    else:
        k += 1
    
os.chdir('..')                                 # salimos del directorio que contiene los .pfds
    
shutil.copy('./pfds/' + tim_out, "./")             # copiamos el archivo .tim a la carpeta

print("observaciones usadas = " + str(j))
print("observaciones eliminadas = " + str(k))


Total de observaciones = 403
observaciones usadas = 358
observaciones eliminadas = 45


In [16]:
#----------------------------------------------------------------------------
# Calculamos residuos
#----------------------------------------------------------------------------  

resfile = "./residues_" + ant + '_pfds_'+ clean + "clean.dat"   # nombre del archivo de residuos
    
parfile = "J0437-4715_JUMPS.par"
#parfile = "J0437-4715.par"
        
subprocess.check_call( "tempo2 >" + resfile + " -us -f " + parfile + " " + tim_out, shell = True)  # calculamos los residuos
    
# rms = np.genfromtxt ( resfile, comments="none", dtype=float, skip_header=21, max_rows=1, usecols=(10) ) # buscamos el RMS

# print(rms)

135.294363226
205.251722096
143.388557841
128.781631446
27.6793851603
192.559987977
21.5983400914
123.384073586
150.981219313
91.6822293661
182.400404604
158.917246134
139.229110851
182.505982361
154.136791567
147.416613698
60.6020980532
134.327092893
206.384275629
76.1859527793
122.551834758
147.363473802
168.774588457
54.4433063228
93.3571755921
95.2174528982
214.308054467
146.504347152
101.272689592
44.1606626288
104.288328648
167.740841066
237.688076079
189.196882346
85.9771799962
203.142148114
7.54837566909
156.736068662
174.209197304
107.74640282
118.642699992
151.148557927
157.778804637
10.9629291797
11.8622806158
140.987587227
19.2753561391
163.220711545
101.614165899
140.05499834
175.994243119
133.606359283
122.759896101
220.265098178
158.716782853
22.8633901456
95.5515669737
116.082061675
113.077174171
191.402255925
108.600555183
134.464316997
191.255843252
166.599705406
89.5083929023
78.9622858081
23.0809774975
129.681170252
101.313855987
100.120579404
32.1483358101
80.87197